<a href="https://colab.research.google.com/github/Juanba98/Algorithmic_Trading/blob/main/equal_weight_S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.3 MB/s 


# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [22]:
import numpy as np
import pandas as pd #for tabular data
import requests #Http request
import xlsxwriter
import math
import json

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [6]:
stocks = pd.read_csv('/content/starter-files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [9]:
from importlib.machinery import SourceFileLoader
from os.path import join
secrets = SourceFileLoader('secrets', '/content/starter-files/secrets.py').load_module()

Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [25]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={secrets.IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 101171416,
 'calculationPrice': 'close',
 'change': -7.48,
 'changePercent': -0.05154,
 'close': 0,
 'closeSource': 'flcifoai',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 149.3,
 'iexCloseTime': 1746473959596,
 'iexLastUpdated': 1714073113400,
 'iexMarketPercent': 15.406432595160966,
 'iexOpen': 153.12,
 'iexOpenTime': 1690491402629,
 'iexRealtimePrice': 145.91,
 'iexRealtimeSize': 103,
 'iexVolume': 2787247,
 'lastTradeTime': 1700715841587,
 'latestPrice': 148.77,
 'latestSource': 'Close',
 'latestTime': 'September 29, 2022',
 'latestUpdate': 1701148040258,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 233425

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [28]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [106]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [107]:
#We must use pd.Series because all row and columns in pd are pd.Series
final_dataframe.append(
    pd.Series(
        [
          symbol,
          price,
          market_cap,
          'N/A'
        ],
        index = my_columns #To assign values to columns
    ),
    
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,ZTS,148.77,2334255967141,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [108]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  #Doing a HTTP request is one of the slowest things that you can do in python
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={secrets.IEX_CLOUD_API_TOKEN}'
  response  = requests.get(api_url)

  if(response.status_code == 404):
    continue
    
  data = response.json()

  final_dataframe = final_dataframe.append(
    pd.Series(
        [
          stock,
          data['latestPrice'],
          data['marketCap'],
          'N/A'
        ],
        index = my_columns #To assign values to columns
    ),
    
    ignore_index=True
  ) 

In [109]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,122.99,36458537889,N/A
1,AAL,12.66,8152531777,N/A
2,AAP,159.85,9842276622,N/A
3,AAPL,143.14,2351823903503,N/A
4,ABBV,145.38,260924079630,N/A
...,...,...,...,...
496,YUM,107.70,30968645862,N/A
497,ZBH,111.42,23008777719,N/A
498,ZBRA,266.82,14287085067,N/A
499,ZION,53.04,8082046367,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [110]:
def chunks(lst,n):
  for i in range (0,len(lst),n):
    # api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={secrets.IEX_CLOUD_API_TOKEN}'
    # response  = requests.get(api_url)

    # if(response.status_code == 404):
    #   continue

    yield lst[i:i+n] #yield is a keyword that is used like return, except the function will return a generator.

In [111]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={secrets.IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    try:
      final_dataframe = final_dataframe.append(
        pd.Series(
            [
              symbol,
              data[symbol]['quote']['latestPrice'],
              data[symbol]['quote']['marketCap'],
              'N/A'
            ],
            index = my_columns #To assign values to columns
        ),
        
        ignore_index=True
      )
    except KeyError as e:
      print(f'Symbol {str(e)} data not found')
  
final_dataframe

Symbol 'DISCA' data not found
Symbol 'HFC' data not found
Symbol 'VIAC' data not found
Symbol 'WLTW' data not found


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.07,37560690977,N/A
1,AAL,12.49,8262207212,N/A
2,AAP,165.89,9613999657,N/A
3,AAPL,146.59,2332731883266,N/A
4,ABBV,148.04,261844401756,N/A
...,...,...,...,...
496,YUM,107.80,31166709688,N/A
497,ZBH,110.95,22941940296,N/A
498,ZBRA,268.59,14155406971,N/A
499,ZION,51.91,8057562161,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [113]:
#@markdown Enter the value of your portfolio.
portfolio_size = 1000000 #@param {type:"integer"}

In [114]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
  final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price']) #[row,column]

final_dataframe 

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.07,37560690977,16
1,AAL,12.49,8262207212,159
2,AAP,165.89,9613999657,12
3,AAPL,146.59,2332731883266,13
4,ABBV,148.04,261844401756,13
...,...,...,...,...
496,YUM,107.80,31166709688,18
497,ZBH,110.95,22941940296,17
498,ZBRA,268.59,14155406971,7
499,ZION,51.91,8057562161,38


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [129]:
writer = pd.ExcelWriter('recomended_trades.xlsx', engine ='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [130]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
      'font_color':background_color,
      'bg_color':font_color,
      'border': 1

    }
)
dollar_format = writer.book.add_format(
    {
      'num_format':'$0.00',
      'font_color':background_color,
      'bg_color':font_color,
      'border': 1

    }
)
integer_format = writer.book.add_format(
    {
      'num_format':'0',
      'font_color':background_color,
      'bg_color':font_color,
      'border': 1

    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [131]:
# writer.sheets['Recomended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recomended Trades'].set_column('B:B', 18, dollar_format)
# writer.sheets['Recomended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recomended Trades'].set_column('D:D', 25, integer_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [132]:
column_formats = {
    'A' : ['Ticker',string_format],
    'B' : ['Stock Price', dollar_format],
    'C' : ['Market Capitalization', dollar_format],
    'D' : ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [133]:
writer.save()

# Referential implementation

*   https://github.com/nickmccullum/algorithmic-trading-python


